### Notebook to Import Postcode ###
We import a postcode dataset found online containing useful information such as latitude, longitude and suburb names (aka locality)

In [2]:
# Setting up Spark Session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()
df = spark.read.csv("../data/postcodes/postcodes.csv", header=True, inferSchema=True)

your 131072x1 screen size is bogus. expect trouble


24/10/17 21:18:35 WARN Utils: Your hostname, DESKTOP-Q5SP5SI resolves to a loopback address: 127.0.1.1; using 172.20.36.110 instead (on interface eth0)
24/10/17 21:18:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Exception in thread "main" java.nio.file.NoSuchFileException: /tmp/tmpv5tk7s8m/connection203236360226178271.info
	at sun.nio.fs.UnixException.translateToIOException(UnixException.java:86)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:102)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:107)
	at sun.nio.fs.UnixFileSystemProvider.newByteChannel(UnixFileSystemProvider.java:214)
	at java.nio.file.Files.newByteChannel(Files.java:361)
	at java.nio.file.Files.createFile(Files.java:632)
	at java.nio.file.TempFileHelper.create(TempFileHelper.java:138)
	at java.nio.file.TempFileHelper.crea

In [2]:
#Set up necessary files
from urllib.request import urlretrieve
import os

# from the current `tute_1` directory, go back two levels to the `MAST30034` directory
output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
if not os.path.exists(output_relative_dir + 'postcodes'):
    os.makedirs(output_relative_dir + 'postcodes')

url = "https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv"
postcodes_output_dir = output_relative_dir + 'postcodes'


# generate output location and filename
output_dir = "postcodes.csv"
output_dir = f"{postcodes_output_dir}/postcodes.csv"

In [4]:
import urllib.request

# bypass Error 403 : Forbidden Access
opener = urllib.request.URLopener()
opener.addheader('User-Agent', 'whatever')  
filename, headers = opener.retrieve(url, output_dir)

We then filtered for Victorian suburbs

In [5]:
vic_df = df.filter(df['state'] == 'VIC')
vic_df.show()

24/10/02 00:03:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+--------+--------------------+-----+-----------+------------+--------------------+-----------------+--------------------+-----+--------------+---+-----------------+------+-----------+------------+-------------+---------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------+-------+-------+--------------------+--------+--------+-----------+----------------+----------+--------+--------------------+------------+-------+-----------+------------------+--------+--------------------+
|   id|postcode|            locality|state|       long|         lat|                  dc|             type|              status|  sa3|       sa3name|sa4|          sa4name|region|Lat_precise|Long_precise|SA1_CODE_2021|  SA1_NAME_2021|SA2_CODE_2021|       SA2_NAME_2021|SA3_CODE_2021|       SA3_NAME_2021|SA4_CODE_2021|       SA4_NAME_2021|RA_2011|RA_2016|RA_2021|        RA_2021_NAME|MMM_2015|MMM_2019|        ced|        altitude|chargezone|phn_c

Unnecessary columns were also removed

In [6]:
columns = ['id','state','dc','type','status','sa3','sa3name','sa4','sa4name','region','Lat_precise',
            'Long_precise','SA1_CODE_2021','SA1_NAME_2021','SA2_CODE_2021','SA2_NAME_2021','SA3_CODE_2021',
            'SA3_NAME_2021','SA4_CODE_2021','SA4_NAME_2021','RA_2011','RA_2016','RA_2021','RA_2021_NAME',
            'MMM_2015','MMM_2019','ced','altitude','chargezone','phn_code','phn_name','lgaregion','lgacode',
            'electorate','electoraterating','sed_code','sed_name']

In [7]:
vic_df = vic_df.drop(*columns)
vic_df.show()

+--------+--------------------+-----------+------------+
|postcode|            locality|       long|         lat|
+--------+--------------------+-----------+------------+
|    3000|           MELBOURNE|144.9825846|-37.81443733|
|    3001|           MELBOURNE|144.9825846|-37.81443733|
|    3002|      EAST MELBOURNE|144.9825846|-37.81443733|
|    3003|      WEST MELBOURNE| 144.949592|  -37.810871|
|    3004|           MELBOURNE|144.9825846|-37.81443733|
|    3004|ST KILDA ROAD CEN...| 144.970161|  -37.844246|
|    3004|ST KILDA ROAD MEL...|    144.976|    -37.8368|
|    3005|  WORLD TRADE CENTRE| 144.950858|  -37.824608|
|    3006|         SOUTH WHARF|144.9520744|-37.82528675|
|    3006|           SOUTHBANK| 144.965926|  -37.823258|
|    3008|           DOCKLANDS| 144.948039|  -37.814719|
|    3010|UNIVERSITY OF MEL...| 144.961351|  -37.796152|
|    3011|           FOOTSCRAY| 144.907953|  -37.807101|
|    3011|              SEDDON| 144.907953|  -37.807101|
|    3011|         SEDDON WEST|

We ensured that there were no federal postcodes in the dataframe

In [9]:
vic_df = vic_df.filter(df['postcode'] < 4000)

Finally, we saved the resulting dataframe in a parquet file

In [10]:
if not os.path.exists("../data/postcodes/postcodes.parquet"):
    vic_df.write.parquet("../data/postcodes/postcodes.parquet")